In [1]:
# --- 1. Install All Dependencies ---

!pip install -q langchain langchain-community langchain-ollama \
               faiss-cpu tiktoken transformers \
               ragas datasets pandas \
               tavily-python \
               yfinance openmeteo-requests requests-cache

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.7/366.7 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.1/167.1 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.9/160.9 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 126.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.8/358.8 

In [2]:
# --- 2. Download and Install Ollama ---
print("\nInstalling Ollama...")
!curl -fsSL https://ollama.com/install.sh | sh
print("✅ Ollama installation complete.")


Installing Ollama...
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
✅ Ollama installation complete.


In [3]:
# --- 3. Start the Ollama Server in the Background ---
import os
import threading
import subprocess

def start_ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=start_ollama)
ollama_thread.start()

In [17]:
!ollama pull nomic-embed-text

In [5]:
!ollama list

NAME    ID    SIZE    MODIFIED 


In [6]:
import os

# --- LangChain-specific Imports ---
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

# --- Ollama-specific Imports ---
from langchain_ollama.chat_models import ChatOllama
from langchain_ollama.embeddings import OllamaEmbeddings


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!ls "/content/drive/MyDrive/dataset"

records


In [9]:
from datasets import load_dataset, load_from_disk

#save_path = "sp500_transcripts_local"

ds = load_from_disk("/content/drive/MyDrive/dataset/records/records")


In [10]:
?ds

In [11]:
from openai import OpenAI
from dotenv import load_dotenv
from pydantic import BaseModel, Field, ValidationError
from typing import List, Optional, Dict
load_dotenv()
from google.colab import userdata
api_key = userdata.get('geminiAPI')


client = None
if not api_key:
    print("⚠️ Set GEMINI_API_KEY in environment or a .env file")
else:
    # Configure the OpenAI client to point to Google's endpoint
    client = OpenAI(
        api_key=api_key,
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
    )
    print("✅ OpenAI client configured for Gemini.")

completion = client.chat.completions.create(
            model="gemini-2.5-flash",
            messages=[{"role": "user", "content": "Hello, world!"}],
            timeout=10
        )

✅ OpenAI client configured for Gemini.


In [12]:
rec = ds[0]

# 2. Print some info about the call
print(f"--- Full Conversation For: {rec['company_name']} ({rec['symbol']}) ---")
print(f"--- Date: {rec['date']} Q{rec['quarter']} {rec['year']} ---\n")

--- Full Conversation For: Agilent Technologies, Inc. (A) ---
--- Date: 2024-11-25 16:30:00 Q4 2024 ---



In [13]:
full_transcript = "\n".join(
    f"{seg['speaker']}: {seg['text']}" for seg in rec["structured_content"]
)

# Now the 'full_transcript' variable holds the entire text in one string.
# You can print it to see:
print(full_transcript)

Operator: Good afternoon. My name is Rob, and I will be your conference operator today. At this time, I would like to welcome everyone to the Agilent Technologies Inc. Fourth Quarter 2024 Earnings Conference Call. All lines have been placed on mute to prevent any background noise. After the speakers' remarks, there will be a question and answer session. If you would like to withdraw your question, again, press star one. Thank you. Parmeet Ahuja, you may begin your conference.
Parmeet Ahuja: Thank you, and welcome everyone to Agilent's conference call for the fourth quarter of fiscal year 2024. I am sure you have seen our press release earlier today regarding our new market-focused organizational structure, which we will talk about in more detail. These changes have no impact on our company's consolidated financial statements. All financial metrics and guidance during this call will be shared under our historical structure. We will provide recast historical segment information to reflec

In [14]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2500,  # Target size for each chunk
    chunk_overlap=200   # How many characters to overlap between chunks - helps mitigate 'sharp' cuts where something is actually part of the next (or previous) chunk
)
docs = [Document(page_content=full_transcript)]
splits = text_splitter.split_documents(docs)

print(f"Split the document into {len(splits)} chunks.")

print("\n--- Example of Chunk 1: ---")
print(splits[0].page_content)
print("\n--- Example of Chunk 2: ---")
print(splits[1].page_content)

Split the document into 33 chunks.

--- Example of Chunk 1: ---
Operator: Good afternoon. My name is Rob, and I will be your conference operator today. At this time, I would like to welcome everyone to the Agilent Technologies Inc. Fourth Quarter 2024 Earnings Conference Call. All lines have been placed on mute to prevent any background noise. After the speakers' remarks, there will be a question and answer session. If you would like to withdraw your question, again, press star one. Thank you. Parmeet Ahuja, you may begin your conference.

--- Example of Chunk 2: ---
Parmeet Ahuja: Thank you, and welcome everyone to Agilent's conference call for the fourth quarter of fiscal year 2024. I am sure you have seen our press release earlier today regarding our new market-focused organizational structure, which we will talk about in more detail. These changes have no impact on our company's consolidated financial statements. All financial metrics and guidance during this call will be shared un

In [15]:
embeddings_model = OllamaEmbeddings(model="nomic-embed-text")

In [18]:
# This one-line command does a lot:
# 1. Calls nomic-embed-text for each of the {len(splits)} chunks
# 2. Gets the embedding (a list of 768 numbers) for each
# 3. Stores them in a FAISS index in RAM

vectorstore = FAISS.from_documents(splits, embeddings_model)

In [19]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [20]:
import json
from pydantic import BaseModel, Field
from typing import List

# --- 1. Your Pydantic Models ---
# (Assuming 'Mention' and 'ReviewFinancial' are defined as before)

class ReviewFinancial(BaseModel):
    """Extracted insights from a financial earnings call transcript."""
    company_name: str = Field(description="The name of the company.")
    symbol: str = Field(description="The company's stock ticker symbol.")

    overall_sentiment: str = Field(
        description="Overall sentiment of the earnings call: 'positive', 'negative', or 'neutral'."
    )

    financials: Dict[str, Optional[str]] = Field(
        description=(
            "Quantitative indicators extracted from the call if mentioned. "
            "Include key performance metrics such as: "
            "revenue_growth (percentage), "
            "eps_change (percentage), "
            "rd_investment_change (percentage), "
            "and outlook_summary (short free-text summary)."
        )
    )

    executives: List[Dict[str, str]] = Field(
        description=(
            "List of executives or speakers mentioned, including their role and general sentiment. "
            "Each entry includes: "
            "name (e.g., 'Sundar Pichai'), "
            "role (e.g., 'CEO', 'CFO'), "
            "and sentiment (positive, negative, neutral)."
        )
    )

    technologies: List[Dict[str, str]] = Field(
        description=(
            "List of technologies mentioned, with contextual financial and strategic insights. "
            "Each entry includes: "
            "technology_name (e.g., 'AI', 'Cloud Computing'), "
            "technology_sentiment (positive, negative, neutral), "
            "investment_action (investing, divesting, scrapping, maintaining, or unclear), "
            "investment_horizon (short-term, long-term, ongoing, or unclear), "
            "profit_outlook (high potential, moderate potential, low potential, or unclear), "
            "rd_investment (numeric string, e.g., '50M USD' or 'up 10%'), "
            "revenue_impact (numeric string, e.g., '+5%' or 'unclear'), "
            "and mentioned_by (executive name if available)."
        )
    )


# --- 2. Your Prompt Template ---
# (Same as before, with {schema} and {transcript_text} placeholders)
prompt_template = '''You are an expert financial analyst API trained to analyze corporate earnings call transcripts.
Your task is to extract structured financial, strategic, and technological insights suitable for a knowledge graph.

**Company:** {company_name} ({symbol})
---
**Transcript to Analyze:**
{transcript_text}
---
**Extraction Guidelines:**

1. **overall_sentiment**
   - Determine the general tone of the call (positive, negative, or neutral) based on financial performance and management commentary.

2. **financials**
   - Extract key quantitative metrics if mentioned, such as:
     - **revenue_growth**: Year-over-year or quarterly growth percentage.
     - **eps_change**: Earnings per share change percentage or direction (e.g., '+5%', 'flat', 'down 3%').
     - **rd_investment_change**: Change in R&D or capital expenditure related to technology.
     - **outlook_summary**: Short summary of guidance (e.g., 'expects moderate growth in Q4').

3. **executives**
   - Identify main speakers (e.g., CEO, CFO) and for each, extract:
     - **name** — Full name of the speaker.
     - **role** — Their title.
     - **sentiment** — General tone of their remarks ('positive', 'negative', or 'neutral').

4. **technologies**
   - Identify each specific technology mentioned and extract:
     - **technology_name** — Name of the technology (1–3 words).
     - **technology_sentiment** — Company's attitude ('positive', 'negative', 'neutral').
     - **investment_action** — 'investing', 'divesting', 'scrapping', 'maintaining', or 'unclear'.
     - **investment_horizon** — 'short-term', 'long-term', 'ongoing', or 'unclear'.
     - **profit_outlook** — 'high potential', 'moderate potential', 'low potential', or 'unclear'.
     - **rd_investment** — Any quantitative R&D or capital expenditure reference (e.g., '50M USD', 'up 10%').
     - **revenue_impact** — Any stated or implied revenue effect (e.g., '+5%', 'neutral', or 'unclear').
     - **mentioned_by** — Name of the executive who discussed the technology, if available.

'''


# First, get the Pydantic schema ready
classification_schema = ReviewFinancial.model_json_schema()
schema_json_string = json.dumps(classification_schema, indent=2)

# Define a query to "focus" the retriever
# This query will find the chunks about the CEO's main remarks
query = "CEO's remarks on Q4 2020 performance and future strategy including concepts as finantial results, technologies and innovation"

# Step 1: RETRIEVE the relevant chunks
# (You would un-comment this line when your 'retriever' is active)
relevant_chunks = retriever.invoke(query)

# --- (Mocking the retriever output for this example) ---
# Let's pretend the retriever returned 3 chunks:
from langchain_core.documents import Document


# Step 2: STITCH the chunks together into one text block
stitched_transcript = "\n\n".join(chunk.page_content for chunk in relevant_chunks)

# Get company_name and symbol from the 'rec' variable available in the kernel state
company_name = rec['company_name']
symbol = rec['symbol']

# Step 3: FORMAT the final prompt
final_prompt = prompt_template.format(
    company_name=company_name,
    symbol=symbol,
    schema=schema_json_string,
    transcript_text=stitched_transcript
)

# Step 4: EXTRACT
# Now, 'final_prompt' is ready to be sent to your LLM (Gemini)
print("--- Final Prompt Sent to LLM ---")
print(final_prompt)

# When you send this 'final_prompt' to Gemini, it will analyze
# ONLY the 3 retrieved chunks and return the JSON you want.

--- Final Prompt Sent to LLM ---
You are an expert financial analyst API trained to analyze corporate earnings call transcripts.
Your task is to extract structured financial, strategic, and technological insights suitable for a knowledge graph.

**Company:** Agilent Technologies, Inc. (A)
---
**Transcript to Analyze:**
Bob McMahon: Thank you, Padraig, and good afternoon, everyone. In my remarks today, I will provide some additional details on fourth quarter revenue and take you through the income statement and other key financial metrics. I will then cover our guidance for fiscal year 2025 and the first quarter of 2025. Unless otherwise noted, my remarks will focus on non-GAAP results. As Padraig said, we are pleased with our Q4 results. Agilent finished the fourth quarter, with core growth in line with our expectations while EPS exceeded our expectations as we executed well against a challenging albeit improving market. Q4 revenue was $1.701 billion, a decline of 0.3% core but a seque

In [21]:
try:
    print("Requesting JSON response from model...")
    completion = client.chat.completions.create(
        model="gemini-2.5-flash-lite",  # Your specified model
        messages=[{"role": "user", "content": final_prompt}],
        response_format={
            "type": "json_schema",
            # Pass the schema *here*, not in the prompt
            "json_schema": {"name": "ReviewFinancial", "strict": True, "schema": classification_schema}
        },
        timeout=30
    )

    choice = completion.choices[0]
    # The response_text is a JSON string *guaranteed* by the API
    response_text = choice.message.content

    # Validate the JSON string directly into your Pydantic model
    result = ReviewFinancial.model_validate_json(response_text)

    # 2. Use the correct V2 method: model_dump_json
    #    Simply remove the 'ensure_ascii' argument.
    print(result.model_dump_json(indent=2))



except Exception as e:
    print(f"\nAn error occurred: {e}")

Requesting JSON response from model...
{
  "company_name": "Agilent Technologies, Inc.",
  "symbol": "A",
  "overall_sentiment": "neutral",
  "financials": {
    "revenue_growth": "-0.3%",
    "eps_change": "exceeded expectations",
    "outlook_summary": "Expects continued recovery throughout fiscal year 2025 with a normalized growth in the back half of the year, potentially driven by China and pharma recovery."
  },
  "executives": [
    {
      "name": "Bob McMahon",
      "role": "Senior Vice President and CFO",
      "sentiment": "positive"
    },
    {
      "name": "Parmeet Ahuja",
      "role": "President and CEO",
      "sentiment": "positive"
    },
    {
      "name": "Padraig McDonnell",
      "role": "President and CEO",
      "sentiment": "positive"
    },
    {
      "name": "Phil Binns",
      "role": "President of the former Life Sciences and Applied Markets Group",
      "sentiment": "neutral"
    },
    {
      "name": "Simon May",
      "role": "President of the newl

**Looping over more queries**

In [22]:

class ReviewFinancial(BaseModel):
    """Extracted insights from a financial earnings call transcript."""
    company_name: str = Field(description="The name of the company.")
    symbol: str = Field(description="The company's stock ticker symbol.")

    overall_sentiment: str = Field(
        description="Overall sentiment of the earnings call: 'positive', 'negative', or 'neutral'."
    )

    financials: Dict[str, Optional[str]] = Field(
        description=(
            "Quantitative indicators extracted from the call if mentioned. "
            "Include key performance metrics such as: "
            "revenue_growth (percentage), "
            "eps_change (percentage), "
            "rd_investment_change (percentage), "
            "and outlook_summary (short free-text summary)."
        )
    )

    executives: List[Dict[str, str]] = Field(
        description=(
            "List of executives or speakers mentioned, including their role and general sentiment. "
            "Each entry includes: "
            "name (e.g., 'Sundar Pichai'), "
            "role (e.g., 'CEO', 'CFO'), "
            "and sentiment (positive, negative, neutral)."
        )
    )

    technologies: List[Dict[str, str]] = Field(
        description=(
            "List of technologies mentioned, with contextual financial and strategic insights. "
            "Each entry includes: "
            "technology_name (e.g., 'AI', 'Cloud Computing'), "
            "technology_sentiment (positive, negative, neutral), "
            "investment_action (investing, divesting, scrapping, maintaining, or unclear), "
            "investment_horizon (short-term, long-term, ongoing, or unclear), "
            "profit_outlook (high potential, moderate potential, low potential, or unclear), "
            "rd_investment (numeric string, e.g., '50M USD' or 'up 10%'), "
            "revenue_impact (numeric string, e.g., '+5%' or 'unclear'), "
            "and mentioned_by (executive name if available)."
        )
    )


# --- 2. Your Prompt Template ---
# (Same as before, with {schema} and {transcript_text} placeholders)
prompt_template = '''You are an expert financial analyst API trained to analyze corporate earnings call transcripts.
Your task is to extract structured financial, strategic, and technological insights suitable for a knowledge graph.

**Company:** {company_name} ({symbol})
---
**Transcript to Analyze:**
{transcript_text}
---
**Extraction Guidelines:**

1. **overall_sentiment**
   - Determine the general tone of the call (positive, negative, or neutral) based on financial performance and management commentary.

2. **financials**
   - Extract key quantitative metrics if mentioned, such as:
     - **revenue_growth**: Year-over-year or quarterly growth percentage.
     - **eps_change**: Earnings per share change percentage or direction (e.g., '+5%', 'flat', 'down 3%').
     - **rd_investment_change**: Change in R&D or capital expenditure related to technology.
     - **outlook_summary**: Short summary of guidance (e.g., 'expects moderate growth in Q4').

3. **executives**
   - Identify main speakers (e.g., CEO, CFO) and for each, extract:
     - **name** — Full name of the speaker.
     - **role** — Their title.
     - **sentiment** — General tone of their remarks ('positive', 'negative', or 'neutral').

4. **technologies**
   - Identify each specific technology mentioned and extract:
     - **technology_name** — Name of the technology (1–3 words).
     - **technology_sentiment** — Company's attitude ('positive', 'negative', 'neutral').
     - **investment_action** — 'investing', 'divesting', 'scrapping', 'maintaining', or 'unclear'.
     - **investment_horizon** — 'short-term', 'long-term', 'ongoing', or 'unclear'.
     - **profit_outlook** — 'high potential', 'moderate potential', 'low potential', or 'unclear'.
     - **rd_investment** — Any quantitative R&D or capital expenditure reference (e.g., '50M USD', 'up 10%').
     - **revenue_impact** — Any stated or implied revenue effect (e.g., '+5%', 'neutral', or 'unclear').
     - **mentioned_by** — Name of the executive who discussed the technology, if available.
'''



In [33]:
from tqdm import tqdm
import time, json

MAX_ROWS_TO_PROCESS = 5
sampled_ds = ds.select(range(401,405))
all_results = []

# Make sure schema is created once
classification_schema = ReviewFinancial.model_json_schema()

for rec in tqdm(sampled_ds, desc="Processing transcripts"):
    try:
        # --- a. Extract metadata ---
        company_name = rec.get("company_name")
        symbol = rec.get("symbol")
        year = rec.get("year")
        quarter = rec.get("quarter")
        date = rec.get("date")

        # --- b. Build the full transcript ---
        full_transcript = "\n".join(
            f"{seg['speaker']}: {seg['text']}" for seg in rec["structured_content"]
        )
        docs = [Document(page_content=full_transcript)]
        splits = text_splitter.split_documents(docs)

        # --- c. Retrieve relevant chunks ---
        vectorstore = FAISS.from_documents(splits, embeddings_model)
        retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
        relevant_chunks = retriever.invoke(query)
        stitched_transcript = "\n\n".join(chunk.page_content for chunk in relevant_chunks)

        # --- d. Format the final prompt ---
        final_prompt_content = prompt_template.format(
            company_name=company_name,
            symbol=symbol,
            transcript_text=stitched_transcript
        )

        # --- e. Run the model ---
        completion = client.chat.completions.create(
            model="gemini-2.5-flash-lite",
            messages=[{"role": "user", "content": final_prompt_content}],
            response_format={
                "type": "json_schema",
                "json_schema": {
                    "name": "ReviewFinancial",
                    "strict": True,
                    "schema": classification_schema
                }
            },
            timeout=30
        )

        response_text = completion.choices[0].message.content

        # --- f. Validate and enrich with metadata ---
        result = ReviewFinancial.model_validate_json(response_text)

        # Convert to dict so we can append metadata
        result_dict = result.model_dump()
        result_dict.update({
            "year": year,
            "quarter": quarter,
            "date": date
        })

        all_results.append(result_dict)

    except Exception as e:
        print(f"\n[ERROR] Failed to process {company_name} ({symbol}): {e}")
    time.sleep(1)


Processing transcripts:   0%|          | 0/4 [00:00<?, ?it/s]


[ERROR] Failed to process ConocoPhillips (COP): Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download


Processing transcripts:  25%|██▌       | 1/4 [00:01<00:03,  1.01s/it]


[ERROR] Failed to process ConocoPhillips (COP): Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download


Processing transcripts:  50%|█████     | 2/4 [00:02<00:02,  1.01s/it]


[ERROR] Failed to process ConocoPhillips (COP): Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download


Processing transcripts:  75%|███████▌  | 3/4 [00:03<00:01,  1.01s/it]


[ERROR] Failed to process ConocoPhillips (COP): Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download


Processing transcripts: 100%|██████████| 4/4 [00:04<00:00,  1.01s/it]


In [29]:
all_results

[{'company_name': 'Bank of America Corporation',
  'symbol': 'BAC',
  'overall_sentiment': 'neutral',
  'financials': {'revenue_growth': 'null',
   'eps_change': 'null',
   'rd_investment_change': 'null',
   'outlook_summary': 'Expects Net Interest Income (NII) to grow in the third and fourth quarters, with Q2 being the trough for the rate cycle. 2024 is seen as a foundational year for future growth.'},
  'executives': [{'name': 'Lee McEntire',
    'role': 'Investor Relations',
    'sentiment': 'neutral'},
   {'name': 'Brian Moynihan', 'role': 'CEO', 'sentiment': 'positive'},
   {'name': 'Alastair Borthwick', 'role': 'CFO', 'sentiment': 'positive'}],
  'technologies': [],
  'year': 2024,
  'quarter': 2,
  'date': '2024-07-16 08:30:00'},
 {'company_name': 'Bank of America Corporation',
  'symbol': 'BAC',
  'overall_sentiment': 'positive',
  'financials': {'revenue_growth': '+35% (Investment Banking), +11% (Investment and brokerage services), +10% (Fees and commissions)',
   'eps_change'

In [ ]:
len(ds)

In [ ]:

# Convert to plain dicts
results_dicts = [r.model_dump() for r in all_results]

# Pretty-print to the notebook
print(json.dumps(results_dicts, indent=2, ensure_ascii=False))

In [24]:
import os, json
from collections import defaultdict

def save_results_grouped_by_company(results, output_dir="results_grouped"):
    """
    Saves one JSON file per company, containing all its transcript analyses across time.
    Uses symbol as the main file identifier.
    """

    os.makedirs(output_dir, exist_ok=True)

    # Group results by company symbol
    grouped = defaultdict(list)
    for r in results:
        symbol = r.get("symbol", "NA")
        grouped[symbol].append(r)

    # Save each company’s results as a list of quarterly entries
    for symbol, entries in grouped.items():
        # Sort by year, then quarter for readability
        entries.sort(key=lambda x: (x.get("year", 0), x.get("quarter", 0)))

        # Use the first company_name for the file
        company_name = entries[0].get("company_name", "unknown").replace("/", "-")

        filename = f"{symbol}_{company_name}.json"
        filepath = os.path.join(output_dir, filename)

        with open(filepath, "w", encoding="utf-8") as f:
            json.dump(entries, f, indent=2, ensure_ascii=False)

    print(f"✅ Saved {len(grouped)} company files in '{output_dir}/'")


In [28]:
save_results_grouped_by_company(all_results)

✅ Saved 51 company files in 'results_grouped/'


In [31]:
import shutil
from google.colab import files

shutil.make_archive("financial_results_grouped400", "zip", "results_grouped")
files.download("financial_results_grouped400.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [34]:
# Path to your grouped results folder
results_path = "results_grouped"

# Collect all JSON files
files = [f for f in os.listdir(results_path) if f.endswith(".json")]

print(f"Found {len(files)} company files.")

Found 101 company files.


In [36]:
unique_technologies = set()

for filename in files:
    with open(os.path.join(results_path, filename), "r", encoding="utf-8") as f:
        data = json.load(f)  # list of quarterly entries

    for entry in data:
        # 'technologies' is a list of dictionaries, not strings
        techs_list = entry.get("technologies") or entry.get("technology") or []
        if isinstance(techs_list, list):
            unique_technologies.update([
                tech_dict['technology_name'].strip()
                for tech_dict in techs_list
                if isinstance(tech_dict, dict) and
                   'technology_name' in tech_dict and
                   isinstance(tech_dict['technology_name'], str) and
                   tech_dict['technology_name'].strip()
            ])

print(f"Found {len(unique_technologies)} unique technologies.")

Found 1380 unique technologies.


In [37]:
unique_technologies = sorted(unique_technologies)
print(f"Total unique technologies: {len(unique_technologies)}")
print(unique_technologies[:50])  # Show first 50 for a preview


Total unique technologies: 1380
['0 carbon ammonia', '1ADM', '383', '3D printing', '3D printing fabrication', '3Dblox', '400', '400 gigabit Ethernet', '400-gig 7080s', '410A', '454B', '48-volt PMEC', '4G', '513', '5G', '5G Infrastructure', '7010D GC/Triple Quad instrument', '737 MAX', '777X', '777X program', '787', '800 gigabit Ethernet', '800-gig AI Ethernet', '8850 GC', '951', 'A2L', 'ACP', 'ADAS', 'ADP Assist', 'AEP', 'AGENT', 'AGS', 'AI', 'AI Accelerators', 'AI Agents', 'AI Assistant', 'AI Era Plan', 'AI Era plan', 'AI Ethernet', 'AI Link', 'AI Networking', 'AI PC', 'AI PCs', 'AI Platform', 'AI infrastructure', 'AI leaf', 'AI platform', 'AI systems', 'AI-Enabled solutions', 'AI-based servers']


In [43]:
import pandas as pd
import os
from google.colab import files # Ensure google.colab.files is imported here

# Path to your grouped results folder
results_path = "results_grouped"

# Collect all JSON files
json_files = [f for f in os.listdir(results_path) if f.endswith(".json")]

# Use a set to ensure uniqueness
unique_technologies = set()

for filename in json_files:
    with open(os.path.join(results_path, filename), "r", encoding="utf-8") as f:
        data = json.load(f)  # list of quarterly entries per company

    for entry in data:
        techs = entry.get("technologies") or entry.get("technology") or []
        if isinstance(techs, list):
            unique_technologies.update([
                tech_dict['technology_name'].strip()
                for tech_dict in techs
                if isinstance(tech_dict, dict) and
                   'technology_name' in tech_dict and
                   isinstance(tech_dict['technology_name'], str) and
                   tech_dict['technology_name'].strip()
            ])

# Convert to sorted list for readability
unique_technologies = sorted(unique_technologies)

# Create a DataFrame with one column
df_tech = pd.DataFrame(list(unique_technologies), columns=["technology"])

# Save to CSV
csv_filename = "unique_technologies.csv"
df_tech.to_csv(csv_filename, index=False, encoding="utf-8")

print(f"✅ Saved {len(df_tech)} unique technologies to '{csv_filename}'")

files.download(csv_filename)

✅ Saved 1380 unique technologies to 'unique_technologies.csv'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [49]:
import pandas as pd
from collections import defaultdict

# 1. Load your unique technologies
df = pd.read_csv("/content/unique_technologies.csv")
terms = df["technology"].astype(str).tolist()

# 2. Define categories + keyword heuristics (all lowercase)
categories = {
    "Networking & Telecom": [
        "5g", "4g", "lte", "ethernet", "fiber", "fibre", "broadband",
        "wifi", "wi-fi", "router", "switch", "telecom", "telco",
        "network", "backhaul"
    ],
    "Cloud, AI & Software": [
        "cloud", "saas", "software", "platform", " ai", "artificial intelligence",
        "machine learning", " ml ", "analytics", "app ", "apps", "application",
        "suite", "crm", "erp", "database", "data lake", "data warehouse",
        "devops"
    ],
    "Semiconductors & Hardware Components": [
        "nanometer", " nm", "chip", "gpu", "cpu", "asic", "fpga",
        "microcontroller", "semiconductor", "wafer", "foundry", "transistor",
        "process node", "7nm", "5nm", "3nm"
    ],
    "Manufacturing, Automation & 3D Printing": [
        "3d printing", "additive", "robot", "robotics", "automation",
        "factory", "manufactur", "industrial", "assembly", "cobot"
    ],
    "Data Center, Compute & Infrastructure": [
        "data center", "datacenter", "server", "rack", "compute",
        "cooling", "thermal", "400 gigabit ethernet", "400g", "400-gig",
        "800 gigabit ethernet", "800g", "800-gig"
    ],
    "Aerospace & Defense": [
        "737", "747", "757", "767", "777", "787", "airbus", "boeing",
        " max", " x program", "space", "satellite", "defense", "missile"
    ],
    "Automotive & Mobility": [
        " ev", "electric vehicle", "autonomous", "adas", "battery",
        "charging", "charger", "drive", "drivetrain", "mobility", "vehicle",
        "automotive"
    ],
    "Energy, Renewables & Storage": [
        "solar", "wind", "pv ", "photovoltaic", "hydrogen", "ammonia",
        "carbon", "decarbon", "renewable", "grid", "nuclear", "fusion",
        "fission", "battery", "storage"
    ],
    "Healthcare, Life Sciences & Biotech": [
        "ngs", "sequencing", "biologics", "biotech", "genomic",
        "genetic", "mrna", "vaccine", "therapy", "therapeutic",
        "oncology", "immuno", "diagnostic", "pathology", "assay",
        "lab ", "laboratory"
    ],
    "Financial & Payments Tech": [
        "payment", "card", "fintech", "wallet", "banking", "lending",
        "credit", "debit", "pos", "terminal"
    ],
    "Retail, E-commerce & Ad Tech": [
        "ad tech", "advertis", "e-commerce", "ecommerce", "retail",
        "marketplace", "shopping"
    ],
    "Logistics, Industrial IoT & Sensors": [
        "sensor", "iot", "telematics", "tracking", "supply chain",
        "logistics", "rfid"
    ],
    # Optional: if you want a dedicated sustainability bucket
    "Sustainability & ESG-related Tech": [
        "esg", "sustainab", "recycl", "green", "carbon", "net zero",
        "climate"
    ],
    # Fallback bucket
    "Other / Miscellaneous": []
}

def classify_term(term: str) -> str:
    """
    Assign a technology term to one of the defined high-level categories.
    If no keyword matches, return 'Other / Miscellaneous'.
    """
    t = term.strip()
    tl = t.lower()

    for cat, keywords in categories.items():
        if cat == "Other / Miscellaneous":
            continue
        for kw in keywords:
            if kw in tl:
                return cat
    return "Other / Miscellaneous"

# 3. Build the dictionary: {category: [technology_1, technology_2, ...]}
grouped_technologies = defaultdict(list)
for term in terms:
    cat = classify_term(term)
    grouped_technologies[cat].append(term)

# Convert back to a normal dict if you like
grouped_technologies = dict(grouped_technologies)

# Quick sanity check: how many per category
for cat, items in grouped_technologies.items():
    print(f"{cat}: {len(items)} technologies")

# If you want to inspect one category, for example:
# grouped_technologies["Cloud, AI & Software"][:30]

Energy, Renewables & Storage: 65 technologies
Other / Miscellaneous: 1017 technologies
Manufacturing, Automation & 3D Printing: 18 technologies
Networking & Telecom: 50 technologies
Data Center, Compute & Infrastructure: 23 technologies
Aerospace & Defense: 13 technologies
Automotive & Mobility: 25 technologies
Cloud, AI & Software: 115 technologies
Semiconductors & Hardware Components: 11 technologies
Retail, E-commerce & Ad Tech: 2 technologies
Healthcare, Life Sciences & Biotech: 17 technologies
Financial & Payments Tech: 14 technologies
Logistics, Industrial IoT & Sensors: 7 technologies
Sustainability & ESG-related Tech: 3 technologies


In [50]:
#show Other / Miscellaneous
grouped_technologies["Other / Miscellaneous"][:100]

['1ADM',
 '383',
 '3Dblox',
 '400',
 '410A',
 '454B',
 '48-volt PMEC',
 '513',
 '7010D GC/Triple Quad instrument',
 '8850 GC',
 '951',
 'A2L',
 'ACP',
 'ADP Assist',
 'AEP',
 'AGENT',
 'AGS',
 'AI',
 'AI Accelerators',
 'AI Agents',
 'AI Assistant',
 'AI Era Plan',
 'AI Era plan',
 'AI Link',
 'AI PC',
 'AI PCs',
 'AI infrastructure',
 'AI leaf',
 'AI systems',
 'AI-Enabled solutions',
 'AI/ML',
 'ALS',
 'AMG 133',
 'AMLD units',
 'API Security',
 'API Solutions',
 'API security',
 'ARPA',
 'ATLAS',
 'AVEIR pacemaker',
 'AWS',
 'Abecma',
 'Absence Insurance',
 'Accelera',
 'Accelera by Cummins',
 'Active ETFs',
 'Active Management',
 'Active Safety',
 'Active Strategies',
 'Adapt Gas Tankless Water Heater',
 'Adaptive computing',
 'Adobe Experience Manager',
 'Adobe GenStudio',
 'Adobe Journey Optimizer',
 'Adobe Workfront Planning',
 'Advanced Materials',
 'Advanced Patient Monitoring',
 'Advanced Process Chemistry',
 'Advanced Safety',
 'Advanced Service Solutions',
 'Advanced diesel